# Lab 4: Game Design in Kivy

## Phase 1

In this lab we will lay the foundation for implementing a game in Kivy. This game will make use of the Game Development concepts learned in class as well as the Kivy drawing commands explored in previous lab. The game we will be implementing is based on agar.io, in which players play as Microorganisms trying to avoid being eaten by larger bodies while consuming smaller bodies. 


You can try out this game here:
[agar.io](http://www.agar.io)

This lab will walk through the foundations of creating the game world and player classes, handling user controls and  implementing collision handling.

## Part 1: Initializing the Game World

To begin, we must set up the game world. For the lab this will be represened as a custom widget class called "World" with the following attributes:

- width = 850 
- height = 480
- Background: A Rectangle of a dark blue color (R = 0, G = 0, B = 0.1, A = 1.0)

The main windows should display the World. We should make the World resize to the size of the window. This can be done using root.size 

Write your world class and .kv file in the cell below.

In [1]:
# write your world class .kv file here.
dotKV = '''\
<World>
    canvas:
        Color:
            rgba: 0, 0, 0.1, 1
        Rectangle:
            pos: (0, 0)
            size: (self.width, self.height)

\
'''
from kivy.lang import Builder
Builder.load_string(dotKV)

from kivy.config import Config
Config.set('graphics', 'fullscreen', '0')
Config.set('graphics', 'width', '850')
Config.set('graphics', 'height', '480')

from kivy.uix.widget import Widget
from kivy.graphics import Rectangle
from kivy.graphics import Color

from kivy.app import App
                 
class World (Widget):
    def __init__(self, **kwargs):
        Widget.__init__(self, **kwargs)
        width = 850
        height = 480
            
class GameApp(App):
    def build(self):
        return World()
        
if __name__ == "__main__":
    GameApp().run()

[INFO   ] [Logger      ] Record log in C:\Users\Quinn\.kivy\logs\kivy_18-03-03_4.txt
[INFO   ] [Kivy        ] v1.10.0
[INFO   ] [Python      ] v3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
[INFO   ] [Factory     ] 194 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil, img_gif (img_ffpyplayer ignored)
[INFO   ] [OSC         ] using <thread> for socket
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] GLEW initialization succeeded
[INFO   ] [GL          ] Backend used <glew>
[INFO   ] [GL          ] OpenGL version <b'4.5.0 - Build 22.20.16.4749'>
[INFO   ] [GL          ] OpenGL vendor <b'Intel'>
[INFO   ] [GL          ] OpenGL renderer <b'Intel(R) HD Graphics 620'>
[INFO   ] [GL          ] OpenGL parsed version: 4, 5
[INFO   ] [GL          ] Shading version <b'4.50 - Build 22.20.16.4749'>
[INFO   ] [GL          ] Texture max size <16384>
[

## Part 2: Creating the local character and enemy class

The next step is to implement a class and widget for the player and the enemies.

A logical approach is to create a custom class and widget for the biological Cells in the game. This class can then be inherrited to create the Player and Enemy class. This is reasonable since the players and the enemies will share a lot of the same code and functionality.  

In the cell below create the ```Cell``` class. It should contain the following members. Use Kivy properties for these types:
```
color: a 4-element list representing the color of this object in the (r,g,b,a) format, where each value is a float from 0.0 to 1.0 
x: the x position of this particular cell
y: the y position of this particular cell
v_x: the velocity in the x direction of this particular cell
v_y: the velocity in the y direction of this particular cell
r: the radius of this particular cell
V_MAX: that maximum velocity in either direction for a cell. Default value should be 2.5
```

In the Kivy code, ensure that when this class is instantiated, it is drawn at the position ```(x,y)``` (centered on the circle) and has a radius of ```r```.

Create ```Player``` and ```Enemy``` as children of this class. For now they do not need any additional properties or methods. We'll add these later.

In the cell below: Add the world code which you create eariler, and add kivy and python code to complete the cell, player and enemy classes.

In [1]:
# write your world class .kv file here.
dotKV = '''\
<World>
    canvas:
        Color:
            rgba: 0, 0, 0.1, 1
        Rectangle:
            pos: (0, 0)
            size: (self.width, self.height)
            
<Cell>
    canvas:
        Color:
            rgba: 1, 0, 0, 1
        Ellipse:
            pos: (self.x - self.r/2, self.y - self.r/2)
            size: (self.r, self.r)
            
        #Debugging to ensure sprite is centered for different size radius
        Color:
            rgba: 0, 1, 0, 1
        Line:
            points: [80, 100, 120, 100]
            width: 1
        Line:
            points: [100, 80, 100, 120]
            width: 1
\
'''
from kivy.lang import Builder
Builder.load_string(dotKV)

from kivy.config import Config
Config.set('graphics', 'fullscreen', '0')
Config.set('graphics', 'width', '850')
Config.set('graphics', 'height', '480')

from kivy.uix.widget import Widget
from kivy.graphics import Rectangle
from kivy.graphics import Color
from kivy.uix.floatlayout import FloatLayout

from kivy.app import App
                 
class World (Widget):
    def __init__(self, **kwargs):
        Widget.__init__(self, **kwargs)
        width = 850
        height = 480
    
class Cell (Widget):
    # Add your cell property declarations here. 
    def __init__(self, **kwargs):
        
        #Pop the kwargs first because Kivy needs to know the variables of the object before the .kv is loaded
        self.color = kwargs.pop('color', [1, 0, 0, 1])
        self.x = kwargs.pop('x', 0)
        self.y = kwargs.pop('y', 0)
        self.v_x = kwargs.pop('v_x', 0)
        self.v_y = kwargs.pop('v_y', 0)
        self.r = kwargs.pop('r', 0)
        self.V_MAX = kwargs.pop('V_MAX', 2.5)
        
        #Call constructor after variables are loaded.
        Widget.__init__(self)

class Player (Cell):
    def __init__(self, **kwargs):
        Cell.__init__(self, **kwargs)

class Enemy (Cell):
    def __init__(self, **kwargs):
        Cell.__init__(self, **kwargs)

class GameApp(App):
    def build(self):
        fl = FloatLayout()
        
        #Create a world instance and a player.
        world = World()
        
        #Tested different types of radius. Players will be centered on their coordinates.
        player = Player(x = 100, y= 100, r = 50)
        
        #Add the world and player to the game
        fl.add_widget(world)
        world.add_widget(player)
        return fl
        
if __name__ == "__main__":
    GameApp().run()

[INFO   ] [Logger      ] Record log in C:\Users\Quinn\.kivy\logs\kivy_18-03-04_26.txt
[INFO   ] [Kivy        ] v1.10.0
[INFO   ] [Python      ] v3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
[INFO   ] [Factory     ] 194 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil, img_gif (img_ffpyplayer ignored)
[INFO   ] [OSC         ] using <thread> for socket
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] GLEW initialization succeeded
[INFO   ] [GL          ] Backend used <glew>
[INFO   ] [GL          ] OpenGL version <b'4.5.0 - Build 22.20.16.4749'>
[INFO   ] [GL          ] OpenGL vendor <b'Intel'>
[INFO   ] [GL          ] OpenGL renderer <b'Intel(R) HD Graphics 620'>
[INFO   ] [GL          ] OpenGL parsed version: 4, 5
[INFO   ] [GL          ] Shading version <b'4.50 - Build 22.20.16.4749'>
[INFO   ] [GL          ] Texture max size <16384>


## Part 3: Adding a Motion Function

Now we will add a motion function to our existing Cell class. We will not be able to invoke this function yet until we create our Game Loop in step 4, but we will need some sort of animation to see that the Game Loop is working. There are 3 portions to this function:

1. Ensure that the velocity is clamped by V_MAX
2. Update the position based on velocity
3. Ensure the position does not got out of bounds, and if so apply a counter velocity corresponding to the cell bouncing off of the walls.

In the following cell add your existing World and Cell classes, and extend the Cell class to include a function called move which applies the three rules explained above:

In [1]:
# write your world class .kv file here.
dotKV = '''\
<World>
    canvas:
        Color:
            rgba: 0, 0, 0.1, 1
        Rectangle:
            pos: (0, 0)
            size: (self.width, self.height)
            
<Cell>
    canvas:
        Color:
            rgba: self.color
        Ellipse:
            pos: (self.x - self.r/2, self.y - self.r/2)
            size: (self.r, self.r)
\
'''
from kivy.lang import Builder
Builder.load_string(dotKV)

from kivy.config import Config
Config.set('graphics', 'fullscreen', '0')
Config.set('graphics', 'width', '850')
Config.set('graphics', 'height', '480')

from kivy.app import App

from kivy.uix.widget import Widget

from kivy.uix.floatlayout import FloatLayout

class World (Widget):
    def __init__(self, **kwargs):
        Widget.__init__(self, **kwargs)
        width = 850
        height = 480
    
class Cell (Widget):
    def __init__(self, **kwargs):
        
        self.color = kwargs.pop('color', [1, 0, 0, 1])
        self.x = kwargs.pop('x', 0)
        self.y = kwargs.pop('y', 0)
        self.v_x = kwargs.pop('v_x', 0)
        self.v_y = kwargs.pop('v_y', 0)
        self.r = kwargs.pop('r', 0)
        self.V_MAX = kwargs.pop('V_MAX', 2.5)
        
        
        Widget.__init__(self)
        
    def move (self):
        #Clamp the velocity to the absolute maximum or minimum
        if(self.v_y > self.V_MAX):
            self.v_y = self.V_MAX
        if(abs(self.v_y) > self.V_MAX) and (self.v_y < 0):
            self.v_y = -self.V_MAX
        if(self.v_x > self.V_MAX):
            self.v_x = self.V_MAX
        if(abs(self.v_x) > self.V_MAX) and (self.v_x < 0):
            self.v_x = -self.V_MAX
        
        #Increase x and y by v_x and v_y respectively
        self.x += self.v_x
        self.y += self.v_y
        
        
        #Check if the position of the objeect is out of bounds.
        #If the object is out of bounds, I am making the object bounce off the edge of the screen.
        if self.x < (0 + self.r):
            self.v_x = -self.v_x
        if self.x > (850 - self.r):
            self.v_x = -self.v_x
        if self.y < (0 + self.r):
            self.v_y = -self.v_y
        if self.y > (480 - self.r):
            self.v_y = -self.v_y

class Player (Cell):
    def __init__(self, **kwargs):
        Cell.__init__(self, **kwargs)

class Enemy (Cell):
    def __init__(self, **kwargs):
        Cell.__init__(self, **kwargs)

class GameApp(App):
    def build(self):
        fl = FloatLayout()
        
        #Create a world instance and a player.
        world = World()
        
        #Tested different types of radius. Players will be centered on their coordinates.
        player = Player(x = 100, y= 100, r = 50)
        
        #Add the world and player to the game
        fl.add_widget(world)
        world.add_widget(player)
        return fl
        
if __name__ == "__main__":
    GameApp().run()

[INFO   ] [Logger      ] Record log in C:\Users\Quinn\.kivy\logs\kivy_18-03-04_30.txt
[INFO   ] [Kivy        ] v1.10.0
[INFO   ] [Python      ] v3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
[INFO   ] [Factory     ] 194 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil, img_gif (img_ffpyplayer ignored)
[INFO   ] [OSC         ] using <thread> for socket
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] GLEW initialization succeeded
[INFO   ] [GL          ] Backend used <glew>
[INFO   ] [GL          ] OpenGL version <b'4.5.0 - Build 22.20.16.4749'>
[INFO   ] [GL          ] OpenGL vendor <b'Intel'>
[INFO   ] [GL          ] OpenGL renderer <b'Intel(R) HD Graphics 620'>
[INFO   ] [GL          ] OpenGL parsed version: 4, 5
[INFO   ] [GL          ] Shading version <b'4.50 - Build 22.20.16.4749'>
[INFO   ] [GL          ] Texture max size <16384>


## Part 4: Refactoring into a Game Class and Adding a Game Loop

In order to test out our motion function, we need to implement the Game Loop. This is a function that will be invoked at repeated intervals. We initialize the game loop using the Clock function. We need to decide the framerate at which the game will run. We'll use 60 frames per second. We must thus call the update function every 60 times per second.

We should take this opportunity to refactor our code into a Game Class. This class should hold references to all of the needed objects for the game, including the world, the player, and a list of enemies. 

Refactoring into a Game Class:

- Create a new class in python called Game
- Write the corresponding Kivy code which includes the World and the Player classes

Additionally, in this class we will create the update function which will run once every 60 frames using the Clock event.

An example of the Clock event is as follows:

```
...
from kivy.clock import Clock
...
Clock.schedule_interval(self.repeat, 1/25)
...
def repeat (self, *args):
...
```

This call will invoke repeat 25 times per second. Be sure to include the ```*args``` argument in the repeat function, as the schedule_interval callback will make use of these parameters. Use this template to create an update function which calls move in the player class.

Finally, to check if the code is working properly, add a v_x of 1.0 and a v_y of 1.0 to the player, and see that it is moving when the game runs. If this is working properly you will a ball moving around in your world and bouncing off of the edges of the walls.

In [1]:
# write your world class .kv file here.
dotKV = '''\
<World>
    canvas:
        Color:
            rgba: 0, 0, 0.1, 1
        Rectangle:
            pos: (0, 0)
            size: (self.width, self.height)
            
<Cell>
    canvas:
        Color:
            rgba: self.color
        Ellipse:
            pos: (self.x - self.r/2, self.y - self.r/2)
            size: (self.r, self.r)
\
'''
from kivy.lang import Builder
Builder.load_string(dotKV)

from kivy.config import Config
Config.set('graphics', 'fullscreen', '0')
Config.set('graphics', 'width', '850')
Config.set('graphics', 'height', '480')

from kivy.app import App

from kivy.uix.widget import Widget

from kivy.uix.floatlayout import FloatLayout

from kivy.graphics import Rectangle, Color

from kivy.clock import Clock

class Game (FloatLayout):
    def __init__(self, **kwargs):
        FloatLayout.__init__(self, **kwargs)
        self.world = World()
        self.add_widget(self.world)
        Clock.schedule_interval(self.oneFrame, 1/60)
        self.counter = 0;
        
    def oneFrame(self, args):
        self.update(self)
    
    def update (self, *args):
        self.world.player.move()
            
class World (Widget):
    def __init__(self, **kwargs):
        Widget.__init__(self, **kwargs)
        self.player = Player(x = 100, y= 100, r = 10, v_y = 2, v_x = 2)
        self.add_widget(self.player)
        width = 850
        height = 480
    
class Cell (Widget):
    # Add your cell property declarations here. 
    def __init__(self, **kwargs):
        self.color = kwargs.pop('color', [1, 0, 0, 1])
        self.x = kwargs.pop('x', 0)
        self.y = kwargs.pop('y', 0)
        self.v_x = kwargs.pop('v_x', 0)
        self.v_y = kwargs.pop('v_y', 0)
        self.r = kwargs.pop('r', 0)
        self.V_MAX = kwargs.pop('V_MAX', 2.5)
        Widget.__init__(self)
        
    def move (self):
        #Check if v_x or v_y is greater than V_MAX or less than negative V_MAX
        #and if so, clamp it to this value
        if(self.v_y > self.V_MAX):
            self.v_y = self.V_MAX
        if(abs(self.v_y) > self.V_MAX) and (self.v_y < 0):
            self.v_y = -self.V_MAX
        if(self.v_x > self.V_MAX):
            self.v_x = self.V_MAX
        if(abs(self.v_x) > self.V_MAX) and (self.v_x < 0):
            self.v_x = -self.V_MAX
               
        #3. Check if the position of the object is out of bounds
        if self.x < (0 + self.r):
            self.v_x = -self.v_x
        if self.x > (850 - self.r):
            self.v_x = -self.v_x
        if self.y < (0 + self.r):
            self.v_y = -self.v_y
        if self.y > (480 - self.r):
            self.v_y = -self.v_y
            
        self.x += self.v_x
        self.y += self.v_y
            
class Player (Cell):
    def __init__(self, **kwargs):
        Cell.__init__(self, **kwargs)

class Enemy (Cell):
    def __init__(self, **kwargs):
        Cell.__init__(self, **kwargs)
        
class GameApp(App):
    def build(self):
        # The game class should now do everything we need to run the Application
        return Game()
        
if __name__ == "__main__":
    GameApp().run()

[INFO   ] [Logger      ] Record log in C:\Users\Quinn\.kivy\logs\kivy_18-03-04_33.txt
[INFO   ] [Kivy        ] v1.10.0
[INFO   ] [Python      ] v3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
[INFO   ] [Factory     ] 194 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil, img_gif (img_ffpyplayer ignored)
[INFO   ] [OSC         ] using <thread> for socket


Purge log fired. Analysing...
Purge finished!


[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] GLEW initialization succeeded
[INFO   ] [GL          ] Backend used <glew>
[INFO   ] [GL          ] OpenGL version <b'4.5.0 - Build 22.20.16.4749'>
[INFO   ] [GL          ] OpenGL vendor <b'Intel'>
[INFO   ] [GL          ] OpenGL renderer <b'Intel(R) HD Graphics 620'>
[INFO   ] [GL          ] OpenGL parsed version: 4, 5
[INFO   ] [GL          ] Shading version <b'4.50 - Build 22.20.16.4749'>
[INFO   ] [GL          ] Texture max size <16384>
[INFO   ] [GL          ] Texture max units <32>
[INFO   ] [Shader      ] fragment shader: <b"WARNING: 0:4: '' :  #version directive missing">
[INFO   ] [Shader      ] vertex shader: <b"WARNING: 0:4: '' :  #version directive missing">
[INFO   ] [Window      ] auto add sdl2 input provider
[INFO   ] [Window      ] virtual keyboard not allowed, single mode, not docked
[INFO   ] [Base        ] Start application main loop
[INFO   ] 

## Part 5: Implementing Input Handling

Our game would be pretty boring if there was no way to control the character. To complete this we need to implement input handling.

We can accomplish this by implementing a new class called InputHanlder. This class should have four boolean properties, one for each of the four directions.

We will need to watch for 4 keys being pressed, up, down, right or left, and the same four keys being released. When the keys are down we need to set the corresponding boolean in the input handler, and when the keys are released we need to clear that boolean value. Finally, we need to check in the Player's "move" function if any of those booleans are set, and if they are we need to apply a change in velocity to the player class. 

Let's walk through these elements one by one. 

We can detect keys being pressed and released using the keyboard functions. A brief code snippet explaining this is shown here:

```
...
from kivy.core.window import Window
...
<in init function>
self._keyboard = Window.request_keyboard(self._keyboard_closed, self)
self._keyboard.bind(on_key_down=self._on_keyboard_down)
self._keyboard.bind(on_key_up=self._on_keyboard_up)
...
<in class>
def _keyboard_closed(self):
    self._keyboard.unbind(on_key_down=self._on_keyboard_down)
    self._keyboard = None

def _on_keyboard_down(self, keyboard, keycode, text, modifiers):
    if keycode[1] == 'up':
        # your code for handling "up" press here...
    if keycode[1] == 'down':
        # your code for handling "down" press here...
    
def _on_keyboard_up(self, keyboard, keycode):
    if keycode[1] == 'up':
        # your code here for handling "up" release here...
```
You should implement this code inside the InputHandler.


Once we have our keys presses correctly implemented, we now need to add a velocity to the player. We can do this by increasing or decreasing the x or y velocity if the key is down by a constant amount of 0.1. Since we have already implenting clamping in the previous section, we do not need to worry about adding it here. Add this functionality here.

If you complete this, then you will have input handling working.

In [1]:

# write your world class .kv file here.
dotKV = '''\
<World>
    canvas:
        Color:
            rgba: 0, 0, 0.1, 1
        Rectangle:
            pos: (0, 0)
            size: (self.width, self.height)
            
<Cell>
    canvas:
        Color:
            rgba: self.color
        Ellipse:
            pos: (self.x - self.r/2, self.y - self.r/2)
            size: (self.r, self.r)
\
'''
from kivy.lang import Builder
Builder.load_string(dotKV)

from kivy.config import Config
Config.set('graphics', 'fullscreen', '0')
Config.set('graphics', 'width', '850')
Config.set('graphics', 'height', '480')

from kivy.app import App

from kivy.uix.widget import Widget

from kivy.uix.floatlayout import FloatLayout

from kivy.graphics import Rectangle, Color

from kivy.clock import Clock

from kivy.core.window import Window

class InputHandler (Widget):
    
    up = False
    down = False
    right = False
    left = False
    
    def __init__ (self, **kwargs):
        Widget.__init__(self, **kwargs)
        self._keyboard = Window.request_keyboard(self._keyboard_closed, self)
        self._keyboard.bind(on_key_down=self._on_keyboard_down)
        self._keyboard.bind(on_key_up=self._on_keyboard_up)
        
    def _keyboard_closed(self):
        self._keyboard.unbind(on_key_down=self._on_keyboard_down)
        self._keyboard = None

    def _on_keyboard_down(self, keyboard, keycode, text, modifiers):
        if keycode[1] == 'up':
            self.up = True
        if keycode[1] == 'down':
            self.down = True
        if keycode[1] == 'left':
            self.left = True
        if keycode[1] == 'right':
            self.right = True

    def _on_keyboard_up(self, keyboard, keycode):
        if keycode[1] == 'up':
            self.up = False
        if keycode[1] == 'down':
            self.down = False
        if keycode[1] == 'left':
            self.left = False
        if keycode[1] == 'right':
            self.right = False

class Game (FloatLayout):
    def __init__(self, **kwargs):
        FloatLayout.__init__(self, **kwargs)
        self.world = World()
        self.add_widget(self.world)
        Clock.schedule_interval(self.oneFrame, 1/60)
        self.counter = 0;
        
    def oneFrame(self, args):
        self.world.update(self)
            
class World (Widget):
    def __init__(self, **kwargs):
        Widget.__init__(self, **kwargs)
        
        self.cells = []
        self.cells = self.cells + [Player(x = 100, y= 100, r = 10)]
        self.add_widget(self.cells[0])
        width = 850
        height = 480
    
    def update(self, *args):
        for c in self.cells:
            c.move()
    
class Cell (Widget):
    
    # Add your cell property declarations here. 
    def __init__(self, **kwargs):
        self.inputHandler = InputHandler()
        self.color = kwargs.pop('color', [1, 0, 0, 1])
        self.x = kwargs.pop('x', 0)
        self.y = kwargs.pop('y', 0)
        self.v_x = kwargs.pop('v_x', 0)
        self.v_y = kwargs.pop('v_y', 0)
        self.r = kwargs.pop('r', 0)
        self.V_MAX = kwargs.pop('V_MAX', 2.5)
        Widget.__init__(self)
        
    def move (self):
        #Update velocity if a key is pressed
        if(self.inputHandler.up):
            self.v_y = self.v_y + 0.1
        if(self.inputHandler.down):
            self.v_y = self.v_y - 0.1
        if(self.inputHandler.left):
            self.v_x = self.v_x - 0.1
        if(self.inputHandler.right):
            self.v_x = self.v_x + 0.1
        
        #Check if v_x or v_y is greater than V_MAX or less than negative V_MAX
        #and if so, clamp it to this value
        if(self.v_y > self.V_MAX):
            self.v_y = self.V_MAX
        if(abs(self.v_y) > self.V_MAX) and (self.v_y < 0):
            self.v_y = -self.V_MAX
        if(self.v_x > self.V_MAX):
            self.v_x = self.V_MAX
        if(abs(self.v_x) > self.V_MAX) and (self.v_x < 0):
            self.v_x = -self.V_MAX
               
        #3. Check if the position of the object is out of bounds
        if self.x < (0 + self.r) and self.v_x < 0:
            self.v_x = 0
        if self.x > (850 - self.r) and self.v_x > 0:
            self.v_x = 0
        if self.y < (0 + self.r) and self.v_y < 0:
            self.v_y = 0
        if self.y > (480 - self.r) and self.v_y > 0:
            self.v_y = 0
            
        self.x += self.v_x
        self.y += self.v_y
            
class Player (Cell):
    def __init__(self, **kwargs):
        Cell.__init__(self, **kwargs)

class Enemy (Cell):
    def __init__(self, **kwargs):
        Cell.__init__(self, **kwargs)
        
class GameApp(App):
    def build(self):
        # The game class should now do everything we need to run the Application
        return Game()
        
if __name__ == "__main__":
    GameApp().run()

[INFO   ] [Logger      ] Record log in C:\Users\Quinn\.kivy\logs\kivy_18-03-04_34.txt
[INFO   ] [Kivy        ] v1.10.0
[INFO   ] [Python      ] v3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
[INFO   ] [Factory     ] 194 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil, img_gif (img_ffpyplayer ignored)
[INFO   ] [OSC         ] using <thread> for socket
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] GLEW initialization succeeded
[INFO   ] [GL          ] Backend used <glew>
[INFO   ] [GL          ] OpenGL version <b'4.5.0 - Build 22.20.16.4749'>
[INFO   ] [GL          ] OpenGL vendor <b'Intel'>
[INFO   ] [GL          ] OpenGL renderer <b'Intel(R) HD Graphics 620'>
[INFO   ] [GL          ] OpenGL parsed version: 4, 5
[INFO   ] [GL          ] Shading version <b'4.50 - Build 22.20.16.4749'>
[INFO   ] [GL          ] Texture max size <16384>


## Part 6: Implementing Collision Resolution

The final step of this lab is to implement collision detection and resolution.

First add an enemy to the game with the following properties:

```
x     = 200
y     = 200
r     = 30
v_x   = 0
v_y   = 0
color = [0.0, 0.0, 1.0, 1.0]
```

You should implement this directly in the python code rather than the Kivy file as we will need to add additional enemies at a later time. 

Next we need to create a collision function for the player and the enemy. 

The purpose of a collision function is to determine if two objects are overlapping. This is found via the following  formula derived from the geometry of the objects:

$ (x_2 - x_1)^2 + (y_2 - y_1)^2 < (r_2 + r_1)^2 $

If this inequality holds, then a collision is occuring. In the event that a collision is detected, we need to handle this in some way. This is known as collision resolution. Typically in this process we want to ensure the overlap is corrected, and that velocities change in some meaningful way to indicate deflection. The simplest way to handle this is to exchange velocities between the two objects, which simulates a mass-ignoring momentum transfer. To accomplish this, simply swap the v_x and v_y variables of the two objects with each other.

Too accomplish this, create a new function in Cell called collisionResolution which takes a pair of Cells and checks for and resolves a collision. Call this function with the Player and the Enemy in the update function, after move.

There are some limitations to this simplistic resolution process, but we can correct these in Phase 2. 

In [1]:

# write your world class .kv file here.
dotKV = '''\
<World>
    canvas:
        Color:
            rgba: 0, 0, 0.1, 1
        Rectangle:
            pos: (0, 0)
            size: (self.width, self.height)
            
<Cell>
    canvas:
        Color:
            rgba: self.color
        Ellipse:
            pos: (self.x - self.r/2, self.y - self.r/2)
            size: (self.r, self.r)
\
'''
from kivy.lang import Builder
Builder.load_string(dotKV)

from kivy.config import Config
Config.set('graphics', 'fullscreen', '0')
Config.set('graphics', 'width', '850')
Config.set('graphics', 'height', '480')

from kivy.app import App

from kivy.uix.widget import Widget

from kivy.uix.floatlayout import FloatLayout

from kivy.graphics import Rectangle, Color

from kivy.clock import Clock

from kivy.core.window import Window

class InputHandler (Widget):
    
    up = False
    down = False
    right = False
    left = False
    
    def __init__ (self, **kwargs):
        Widget.__init__(self, **kwargs)
        self._keyboard = Window.request_keyboard(self._keyboard_closed, self)
        self._keyboard.bind(on_key_down=self._on_keyboard_down)
        self._keyboard.bind(on_key_up=self._on_keyboard_up)
        
    def _keyboard_closed(self):
        self._keyboard.unbind(on_key_down=self._on_keyboard_down)
        self._keyboard = None

    def _on_keyboard_down(self, keyboard, keycode, text, modifiers):
        if keycode[1] == 'up':
            self.up = True
        if keycode[1] == 'down':
            self.down = True
        if keycode[1] == 'left':
            self.left = True
        if keycode[1] == 'right':
            self.right = True

    def _on_keyboard_up(self, keyboard, keycode):
        if keycode[1] == 'up':
            self.up = False
        if keycode[1] == 'down':
            self.down = False
        if keycode[1] == 'left':
            self.left = False
        if keycode[1] == 'right':
            self.right = False

class Game (FloatLayout):
    def __init__(self, **kwargs):
        FloatLayout.__init__(self, **kwargs)
        self.world = World()
        self.add_widget(self.world)
        Clock.schedule_interval(self.oneFrame, 1/60)
        self.counter = 0;
        
    def oneFrame(self, args):
        self.world.update(self)
            
class World (Widget):
    def __init__(self, **kwargs):
        Widget.__init__(self, **kwargs)
        
        self.cells = []
        self.cells = self.cells + [Player(x = 100, y= 100, r = 10)]
        self.cells = self.cells + [Enemy(x = 200, y = 200, r = 30, color = [0.0, 0.0, 1.0, 1.0])]
        for c in self.cells:
            self.add_widget(c)
            
        width = 850
        height = 480
    
    def update(self, *args):                    
        for c in self.cells:
            c.move()
            for other in self.cells:
                if(other is not c):
                    c.collisionResolution(other)
    
class Cell (Widget):
    
    # Add your cell property declarations here. 
    def __init__(self, **kwargs):
        self.color = kwargs.pop('color', [1, 0, 0, 1])
        self.x = kwargs.pop('x', 0)
        self.y = kwargs.pop('y', 0)
        self.v_x = kwargs.pop('v_x', 0)
        self.v_y = kwargs.pop('v_y', 0)
        self.r = kwargs.pop('r', 0)
        self.V_MAX = kwargs.pop('V_MAX', 2.5)
        Widget.__init__(self)
        
    def update (self, *args):
        self.player.move()
        self.collisionResolution(self, )
        
    def collisionResolution(self, obj1):
        if (pow(self.x - obj1.x, 2) + pow(self.y - obj1.y, 2) < pow(self.r - obj1.r, 2)):
            v_x1 = obj1.v_x
            v_y1 = obj1.v_y
            obj1.v_x = self.v_y
            obj1.v_y = self.v_x
            self.v_x = v_y1
            self.v_y = v_x1
        
    def move (self):        
        #Check if v_x or v_y is greater than V_MAX or less than negative V_MAX
        #and if so, clamp it to this value
        if(self.v_y > self.V_MAX):
            self.v_y = self.V_MAX
        if(abs(self.v_y) > self.V_MAX) and (self.v_y < 0):
            self.v_y = -self.V_MAX
        if(self.v_x > self.V_MAX):
            self.v_x = self.V_MAX
        if(abs(self.v_x) > self.V_MAX) and (self.v_x < 0):
            self.v_x = -self.V_MAX
            
        self.x += self.v_x
        self.y += self.v_y
            
class Player (Cell):
    def __init__(self, **kwargs):
        Cell.__init__(self, **kwargs)
        self.inputHandler = InputHandler()
        
    def move(self):
        #Call parnets move function
        Cell.move(self)
        
        #Override the movement of players so that only players move with the input handler
        if(self.inputHandler.up):
            self.v_y = self.v_y + 0.1
        if(self.inputHandler.down):
            self.v_y = self.v_y - 0.1
        if(self.inputHandler.left):
            self.v_x = self.v_x - 0.1
        if(self.inputHandler.right):
            self.v_x = self.v_x + 0.1
            
        #Override player velocity movement so players dont bounce off walls
        if self.x < (0 + self.r) and self.v_x < 0:
            self.v_x = 0
        if self.x > (850 - self.r) and self.v_x > 0:
            self.v_x = 0
        if self.y < (0 + self.r) and self.v_y < 0:
            self.v_y = 0
        if self.y > (480 - self.r) and self.v_y > 0:
            self.v_y = 0

class Enemy (Cell):
    def __init__(self, **kwargs):
        Cell.__init__(self, **kwargs)
        
    def move(self):
        #Call parent's move function
        Cell.move(self)
    
        #Override the velocity of enemies to make the enemies bounce off the walls
        if self.x < (0 + self.r) and self.v_x < 0:
            self.v_x = -self.v_x
        if self.x > (850 - self.r) and self.v_x > 0:
            self.v_x = -self.v_x
        if self.y < (0 + self.r) and self.v_y < 0:
            self.v_y = -self.v_y
        if self.y > (480 - self.r) and self.v_y > 0:
            self.v_y = -self.v_y
        
class GameApp(App):
    def build(self):
        # The game class should now do everything we need to run the Application
        return Game()
        
if __name__ == "__main__":
    GameApp().run()

[INFO   ] [Logger      ] Record log in C:\Users\Quinn\.kivy\logs\kivy_18-03-04_38.txt
[INFO   ] [Kivy        ] v1.10.0
[INFO   ] [Python      ] v3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
[INFO   ] [Factory     ] 194 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil, img_gif (img_ffpyplayer ignored)
[INFO   ] [OSC         ] using <thread> for socket
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] GLEW initialization succeeded
[INFO   ] [GL          ] Backend used <glew>
[INFO   ] [GL          ] OpenGL version <b'4.5.0 - Build 22.20.16.4749'>
[INFO   ] [GL          ] OpenGL vendor <b'Intel'>
[INFO   ] [GL          ] OpenGL renderer <b'Intel(R) HD Graphics 620'>
[INFO   ] [GL          ] OpenGL parsed version: 4, 5
[INFO   ] [GL          ] Shading version <b'4.50 - Build 22.20.16.4749'>
[INFO   ] [GL          ] Texture max size <16384>


Congratulations. If you have made it this far you now have a working, if slightly simplistic, physics engine. In the Assignment phase you will continue work on the game, including consuming enemies, setting win and loss conditions, and adding enemies procedurally.